# Object Detection - FPN(Feature Pyramid Network)
- Paper : (2016)FPN-Feature Pyramid Networks for Object Detection

## 목적
- 과거엔 작은 객체를 잘 못잡는다가 많은 이슈
- 다양한 크기의 객체를 더 잘 탐지하기 위해서
- Backbone을 거치면서 나온 Feature map들을 Neck으로 보냄.

## 과거의 작은 객체를 잘잡기 위한 노력
1) 큰 이미지를 Resize를 통해, 각 이미지에서 Feature Map 추출
2) 이미지 Conv간 나오는 중간의 Feature map 활용 = SSD

## Neck의 역할
- Low level 에서는 큰 크기의 Object를 잡고 => 모서리 위주의 Feature map 형성
- High level에서는 작은 물체를 잘 잡음 => 더 구체적인 정보를 잡을 것
- High level에서 뽑아오는 Feature map을 Conv를 통해 Low level로 보내면서 Feature map을 섞음

<hr>



## Feature Pyramid Network
- Bottom-up & Top-down 방식
    - **Botton-up(Conv: Low Level => High level)
    - Top-down pathway(Lateral connections: Feature map 2배 upsampling + 1x1 conv를 통해 bottom-up에서의 정보를 합침(element-wise))
- **low-resolution 및 high-resolution을 묶어서 사용하는 방식**
- 상위 레벨의 특징을 재사용하여 Multi-scale 특징들을 효율적으로 사용
- **Backbone => Neck => 여러 Neck에서 나온 Feature map을 RPN 진행(Anchor Box기반의 Sliding Window + Bbox Regression + Classification) => 매우 많은 ROI들이 생성 k를 통해 ROI Projection 대상 구함**
![FPN_Network](img/FPN_Network.png)

<hr>

## Bottom-up pathway
- 상향식 과정 간 Backbone - Resnet
- 각 단계의 마지막 레이어의 출력을 특징 맵의 Reference Set 으로 선택
    - 해당 넘겨주는 위치는 이미지가 절반으로 줄 때
- 피라미드를 풍부하게 생성하며, 각 단계의 가장 깊은 레이어에는 가장 영향력 있는 특징이 있어야 함

## Top-down pathway and lateral connections
- 하향식 과정은 많은 의미 정보들을 가지고 있는 특징 맵을 2배로 업샘플링(Up-sampling)하여, 더 높은 해상도의 이미지를 만드는 역할을 수행
- Nearest Neighbor Upsampling (2468 => 2222444466668888)

- **Top-down 중 skip-connection을 통해 같은 사이즈의 bottom-up Feature map과 합쳐서 손실된 정보를 보충**
- **1x1 convolution으로 채널수를 맞춰준다음 element-wise 진행**
- Top-down 과정에서 매 레이어 마다 classfifier, regressor 가 적용
- 같은 classfifier / regressor 를 사용하기 때문에 element-wise 해온 채널이 256으로 전부 같음.

![fpn_pipeline](img/fpn_pipeline.PNG)

<hr>

### RPN - 1x1 Conv
- BboxRegression과 classification 계산 
- input 차원 N x N x 4 => 1x1 conv 2개를 하는 것은 결국 아래연산과 같음
- 즉 1x1 conv는 FC Layer와 같다고 할 수 있음.
![rpn_1x1conv](img/rpn_1x1conv.png)

### 한계점
- 정보전달의 Path가 짧아 low-layer의 특징들의 전파에 아쉬움이 있음.
    - PANet에서 한번더 Botton-up을 함으로 서 해결
    
<hr>

## 요약
- 여러 Scale의 물체를 탐지하기 위해 설계

- Bottom-up : 일반 Conv 과정 (ResNet)
- Top-down : up-sampling 정보 + skip-connection을 통해 bottom-up 정보를 합쳐, 손실된 정보를 보충

- 1x1 conv를 통한 채널수 맞추고 element-wise, 각 모든 레벨에서 독립적으로 잘 예측 할 수 있는 Architecture를 만들며, 각 level마다 classifier와 regressor가 존재

- **큰 박스보다는 small box의 AP를 많이 향상 시킴**

## 참고문헌
- https://eehoeskrap.tistory.com/300
- https://velog.io/@haejoo/Feature-Pyramid-Networks-for-Object-Detection-%EB%85%BC%EB%AC%B8-%EC%A0%95%EB%A6%AC